# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/13_systemes_recommandation/13_demo_neural_recommenders.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '13_demo_neural_recommenders.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 14 - Démonstration : Neural Recommenders

Ce notebook explore les approches de **Deep Learning** pour les systèmes de recommandation :

1. **Neural Collaborative Filtering (NCF)**
2. **Autoencoders pour Recommandation**
3. **Two-Tower Model**
4. **Évaluation avec métriques de ranking** (Precision@K, Recall@K, NDCG@K)
5. **Comparaison avec approches classiques**

Dataset : **MovieLens 100K**

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. Chargement et Préparation des Données

In [ ]:
# Charger MovieLens 100K
from surprise import Dataset

data = Dataset.load_builtin('ml-100k')
ratings_df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])

# Créer des IDs numériques consécutifs
user_ids = ratings_df['user_id'].unique()
item_ids = ratings_df['item_id'].unique()

user_id_map = {uid: idx for idx, uid in enumerate(user_ids)}
item_id_map = {iid: idx for idx, iid in enumerate(item_ids)}

ratings_df['user_idx'] = ratings_df['user_id'].map(user_id_map)
ratings_df['item_idx'] = ratings_df['item_id'].map(item_id_map)

n_users = len(user_ids)
n_items = len(item_ids)

print(f"Nombre d'utilisateurs: {n_users}")
print(f"Nombre de films: {n_items}")
print(f"Nombre de ratings: {len(ratings_df)}")

# Train/Validation/Test split
train_df, temp_df = train_test_split(ratings_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"\nTrain: {len(train_df)} | Validation: {len(val_df)} | Test: {len(test_df)}")

In [ ]:
# PyTorch Dataset
class RatingDataset(Dataset):
    def __init__(self, df):
        self.users = torch.LongTensor(df['user_idx'].values)
        self.items = torch.LongTensor(df['item_idx'].values)
        self.ratings = torch.FloatTensor(df['rating'].values)
    
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]

# Create DataLoaders
batch_size = 512

train_dataset = RatingDataset(train_df)
val_dataset = RatingDataset(val_df)
test_dataset = RatingDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

## 2. Neural Collaborative Filtering (NCF)

Architecture combinant embeddings + MLP pour modéliser les interactions user-item.

In [ ]:
class NCF(nn.Module):
    def __init__(self, n_users, n_items, embedding_dim=64, hidden_layers=[128, 64, 32], dropout=0.2):
        super(NCF, self).__init__()
        
        # Embeddings
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        
        # MLP layers
        layers = []
        input_dim = embedding_dim * 2
        
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_dim = hidden_dim
        
        self.mlp = nn.Sequential(*layers)
        self.output = nn.Linear(hidden_layers[-1], 1)
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        nn.init.normal_(self.user_embedding.weight, std=0.01)
        nn.init.normal_(self.item_embedding.weight, std=0.01)
    
    def forward(self, user_ids, item_ids):
        # Embeddings
        user_emb = self.user_embedding(user_ids)  # (batch, embedding_dim)
        item_emb = self.item_embedding(item_ids)  # (batch, embedding_dim)
        
        # Concatenate
        x = torch.cat([user_emb, item_emb], dim=-1)  # (batch, 2*embedding_dim)
        
        # MLP
        x = self.mlp(x)
        
        # Output
        rating = self.output(x).squeeze()  # (batch,)
        return rating

# Instantiate model
ncf_model = NCF(n_users, n_items, embedding_dim=64, hidden_layers=[128, 64, 32], dropout=0.2)
ncf_model = ncf_model.to(device)

print(ncf_model)
print(f"\nTotal parameters: {sum(p.numel() for p in ncf_model.parameters()):,}")

In [ ]:
# Training function
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for user_ids, item_ids, ratings in loader:
        user_ids = user_ids.to(device)
        item_ids = item_ids.to(device)
        ratings = ratings.to(device)
        
        # Forward
        predictions = model(user_ids, item_ids)
        loss = criterion(predictions, ratings)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

def evaluate(model, loader, device):
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for user_ids, item_ids, ratings in loader:
            user_ids = user_ids.to(device)
            item_ids = item_ids.to(device)
            
            predictions = model(user_ids, item_ids)
            
            all_preds.extend(predictions.cpu().numpy())
            all_targets.extend(ratings.numpy())
    
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    
    rmse = np.sqrt(mean_squared_error(all_targets, all_preds))
    mae = mean_absolute_error(all_targets, all_preds)
    
    return rmse, mae, all_preds, all_targets

In [ ]:
# Train NCF
criterion = nn.MSELoss()
optimizer = optim.Adam(ncf_model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

n_epochs = 15
train_losses = []
val_rmses = []

print("=== Training NCF ===")
for epoch in range(n_epochs):
    train_loss = train_epoch(ncf_model, train_loader, criterion, optimizer, device)
    val_rmse, val_mae, _, _ = evaluate(ncf_model, val_loader, device)
    
    train_losses.append(train_loss)
    val_rmses.append(val_rmse)
    
    scheduler.step(val_rmse)
    
    if (epoch + 1) % 3 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {train_loss:.4f} | Val RMSE: {val_rmse:.4f} | Val MAE: {val_mae:.4f}")

print("\nTraining completed!")

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(train_losses, label='Train Loss', marker='o')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].plot(val_rmses, label='Validation RMSE', marker='o', color='coral')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('RMSE')
axes[1].set_title('Validation RMSE')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate on test set
test_rmse_ncf, test_mae_ncf, test_preds_ncf, test_targets = evaluate(ncf_model, test_loader, device)

print("=== NCF Test Results ===")
print(f"RMSE: {test_rmse_ncf:.4f}")
print(f"MAE: {test_mae_ncf:.4f}")

# Scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(test_targets, test_preds_ncf, alpha=0.3, s=20)
plt.plot([1, 5], [1, 5], 'r--', lw=2, label='Perfect prediction')
plt.xlabel('Ratings Réels')
plt.ylabel('Ratings Prédits (NCF)')
plt.title(f'NCF: Prédictions vs Réalité\nRMSE = {test_rmse_ncf:.3f}')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## 3. Autoencoder pour Recommandation (AutoRec)

Autoencoder qui reconstruit le vecteur de ratings d'un utilisateur.

In [ ]:
class AutoRec(nn.Module):
    def __init__(self, n_items, hidden_dim=256, dropout=0.3):
        super(AutoRec, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(n_items, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, n_items)
        )
    
    def forward(self, ratings):
        # ratings: (batch, n_items) avec 0 pour items non ratés
        h = self.encoder(ratings)  # (batch, hidden_dim)
        reconstructed = self.decoder(h)  # (batch, n_items)
        return reconstructed

autorec_model = AutoRec(n_items, hidden_dim=256, dropout=0.3)
autorec_model = autorec_model.to(device)

print(autorec_model)
print(f"\nTotal parameters: {sum(p.numel() for p in autorec_model.parameters()):,}")

In [ ]:
# Create user-item matrices for AutoRec
def create_user_vectors(df, n_users, n_items):
    """Create user vectors (n_users, n_items) with ratings."""
    user_vectors = np.zeros((n_users, n_items))
    for _, row in df.iterrows():
        user_vectors[int(row['user_idx']), int(row['item_idx'])] = row['rating']
    return user_vectors

train_user_vectors = create_user_vectors(train_df, n_users, n_items)
val_user_vectors = create_user_vectors(val_df, n_users, n_items)
test_user_vectors = create_user_vectors(test_df, n_users, n_items)

# Create masks (1 where rated, 0 otherwise)
train_mask = (train_user_vectors > 0).astype(np.float32)
val_mask = (val_user_vectors > 0).astype(np.float32)
test_mask = (test_user_vectors > 0).astype(np.float32)

# Convert to tensors
train_user_vectors = torch.FloatTensor(train_user_vectors).to(device)
train_mask = torch.FloatTensor(train_mask).to(device)
val_user_vectors = torch.FloatTensor(val_user_vectors).to(device)
val_mask = torch.FloatTensor(val_mask).to(device)
test_user_vectors = torch.FloatTensor(test_user_vectors).to(device)
test_mask = torch.FloatTensor(test_mask).to(device)

print(f"Train user vectors shape: {train_user_vectors.shape}")

In [ ]:
# Training AutoRec
def train_autorec(model, user_vectors, mask, criterion, optimizer):
    model.train()
    reconstructed = model(user_vectors)
    
    # Loss only on observed ratings
    loss = criterion(reconstructed * mask, user_vectors * mask)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()

def evaluate_autorec(model, user_vectors, mask):
    model.eval()
    with torch.no_grad():
        reconstructed = model(user_vectors)
        
        # Extract predictions for observed ratings
        mask_np = mask.cpu().numpy()
        preds = reconstructed.cpu().numpy()[mask_np > 0]
        targets = user_vectors.cpu().numpy()[mask_np > 0]
        
        rmse = np.sqrt(mean_squared_error(targets, preds))
        mae = mean_absolute_error(targets, preds)
    
    return rmse, mae

# Train
criterion_ae = nn.MSELoss()
optimizer_ae = optim.Adam(autorec_model.parameters(), lr=0.001, weight_decay=1e-5)

n_epochs_ae = 50
train_losses_ae = []
val_rmses_ae = []

print("=== Training AutoRec ===")
for epoch in range(n_epochs_ae):
    train_loss = train_autorec(autorec_model, train_user_vectors, train_mask, criterion_ae, optimizer_ae)
    val_rmse, val_mae = evaluate_autorec(autorec_model, val_user_vectors, val_mask)
    
    train_losses_ae.append(train_loss)
    val_rmses_ae.append(val_rmse)
    
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{n_epochs_ae} | Train Loss: {train_loss:.4f} | Val RMSE: {val_rmse:.4f} | Val MAE: {val_mae:.4f}")

print("\nTraining completed!")

In [ ]:
# Evaluate on test set
test_rmse_ae, test_mae_ae = evaluate_autorec(autorec_model, test_user_vectors, test_mask)

print("=== AutoRec Test Results ===")
print(f"RMSE: {test_rmse_ae:.4f}")
print(f"MAE: {test_mae_ae:.4f}")

## 4. Two-Tower Model

Architecture avec deux tours séparées pour utilisateurs et items.

In [ ]:
class TwoTowerModel(nn.Module):
    def __init__(self, n_users, n_items, embedding_dim=64, hidden_dim=128):
        super(TwoTowerModel, self).__init__()
        
        # User Tower
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.user_tower = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, embedding_dim)
        )
        
        # Item Tower
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        self.item_tower = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, embedding_dim)
        )
        
        self._init_weights()
    
    def _init_weights(self):
        nn.init.normal_(self.user_embedding.weight, std=0.01)
        nn.init.normal_(self.item_embedding.weight, std=0.01)
    
    def forward(self, user_ids, item_ids):
        # User embedding
        user_emb = self.user_embedding(user_ids)  # (batch, embedding_dim)
        user_vec = self.user_tower(user_emb)  # (batch, embedding_dim)
        
        # Item embedding
        item_emb = self.item_embedding(item_ids)  # (batch, embedding_dim)
        item_vec = self.item_tower(item_emb)  # (batch, embedding_dim)
        
        # Dot product
        scores = torch.sum(user_vec * item_vec, dim=-1)  # (batch,)
        return scores
    
    def get_user_embedding(self, user_ids):
        user_emb = self.user_embedding(user_ids)
        return self.user_tower(user_emb)
    
    def get_item_embedding(self, item_ids):
        item_emb = self.item_embedding(item_ids)
        return self.item_tower(item_emb)

two_tower_model = TwoTowerModel(n_users, n_items, embedding_dim=64, hidden_dim=128)
two_tower_model = two_tower_model.to(device)

print(two_tower_model)
print(f"\nTotal parameters: {sum(p.numel() for p in two_tower_model.parameters()):,}")

In [ ]:
# Train Two-Tower
criterion_tt = nn.MSELoss()
optimizer_tt = optim.Adam(two_tower_model.parameters(), lr=0.001, weight_decay=1e-5)

n_epochs_tt = 15
train_losses_tt = []
val_rmses_tt = []

print("=== Training Two-Tower Model ===")
for epoch in range(n_epochs_tt):
    train_loss = train_epoch(two_tower_model, train_loader, criterion_tt, optimizer_tt, device)
    val_rmse, val_mae, _, _ = evaluate(two_tower_model, val_loader, device)
    
    train_losses_tt.append(train_loss)
    val_rmses_tt.append(val_rmse)
    
    if (epoch + 1) % 3 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{n_epochs_tt} | Train Loss: {train_loss:.4f} | Val RMSE: {val_rmse:.4f} | Val MAE: {val_mae:.4f}")

print("\nTraining completed!")

In [ ]:
# Evaluate on test set
test_rmse_tt, test_mae_tt, test_preds_tt, _ = evaluate(two_tower_model, test_loader, device)

print("=== Two-Tower Test Results ===")
print(f"RMSE: {test_rmse_tt:.4f}")
print(f"MAE: {test_mae_tt:.4f}")

## 5. Métriques de Ranking (Precision@K, Recall@K, NDCG@K)

Évaluons les modèles avec des métriques de ranking pour les recommandations top-K.

In [ ]:
def precision_at_k(recommendations, relevant_items, k):
    """Precision@K: proportion d'items pertinents dans les K recommandés."""
    top_k = recommendations[:k]
    relevant_in_top_k = len(set(top_k) & relevant_items)
    return relevant_in_top_k / k if k > 0 else 0

def recall_at_k(recommendations, relevant_items, k):
    """Recall@K: proportion d'items pertinents retrouvés dans les K recommandés."""
    top_k = recommendations[:k]
    relevant_in_top_k = len(set(top_k) & relevant_items)
    return relevant_in_top_k / len(relevant_items) if len(relevant_items) > 0 else 0

def ndcg_at_k(recommendations, true_relevances, k):
    """NDCG@K: Normalized Discounted Cumulative Gain."""
    # DCG
    dcg = 0
    for i, item_id in enumerate(recommendations[:k]):
        relevance = true_relevances.get(item_id, 0)
        dcg += (2**relevance - 1) / np.log2(i + 2)  # i+2 car log2(1) = 0
    
    # IDCG (ideal DCG avec tri parfait)
    ideal_relevances = sorted(true_relevances.values(), reverse=True)[:k]
    idcg = sum((2**rel - 1) / np.log2(i + 2) for i, rel in enumerate(ideal_relevances))
    
    return dcg / idcg if idcg > 0 else 0

def evaluate_ranking(model, test_df, train_df, k_list=[5, 10, 20], threshold=4.0, n_users_sample=100):
    """
    Évaluer les métriques de ranking pour un échantillon d'utilisateurs.
    
    Args:
        model: modèle de recommandation
        test_df: DataFrame de test
        train_df: DataFrame de train
        k_list: liste des valeurs de K
        threshold: rating >= threshold est considéré comme pertinent
        n_users_sample: nombre d'utilisateurs à échantillonner
    """
    model.eval()
    
    # Échantillonner des utilisateurs ayant assez de ratings dans test
    user_test_counts = test_df.groupby('user_idx').size()
    users_with_data = user_test_counts[user_test_counts >= 5].index.tolist()
    sample_users = np.random.choice(users_with_data, min(n_users_sample, len(users_with_data)), replace=False)
    
    results = {k: {'precision': [], 'recall': [], 'ndcg': []} for k in k_list}
    
    for user_idx in sample_users:
        # Items ratés dans train (à exclure des recommandations)
        train_items = set(train_df[train_df['user_idx'] == user_idx]['item_idx'])
        
        # Items pertinents dans test (rating >= threshold)
        test_user_df = test_df[test_df['user_idx'] == user_idx]
        relevant_items = set(test_user_df[test_user_df['rating'] >= threshold]['item_idx'])
        
        if len(relevant_items) == 0:
            continue
        
        # Créer dict des relevances (ratings)
        true_relevances = test_user_df.set_index('item_idx')['rating'].to_dict()
        
        # Prédire pour tous les items candidats (non dans train)
        candidate_items = [i for i in range(n_items) if i not in train_items]
        
        user_tensor = torch.LongTensor([user_idx] * len(candidate_items)).to(device)
        item_tensor = torch.LongTensor(candidate_items).to(device)
        
        with torch.no_grad():
            predictions = model(user_tensor, item_tensor).cpu().numpy()
        
        # Trier par score décroissant
        sorted_indices = np.argsort(predictions)[::-1]
        recommendations = [candidate_items[i] for i in sorted_indices]
        
        # Calculer les métriques pour chaque K
        for k in k_list:
            prec = precision_at_k(recommendations, relevant_items, k)
            rec = recall_at_k(recommendations, relevant_items, k)
            ndcg = ndcg_at_k(recommendations, true_relevances, k)
            
            results[k]['precision'].append(prec)
            results[k]['recall'].append(rec)
            results[k]['ndcg'].append(ndcg)
    
    # Moyennes
    avg_results = {}
    for k in k_list:
        avg_results[k] = {
            'precision': np.mean(results[k]['precision']),
            'recall': np.mean(results[k]['recall']),
            'ndcg': np.mean(results[k]['ndcg'])
        }
    
    return avg_results

In [ ]:
# Évaluer NCF avec métriques de ranking
print("\n=== Évaluation Ranking NCF (cela peut prendre quelques minutes) ===")
ranking_results_ncf = evaluate_ranking(ncf_model, test_df, train_df, k_list=[5, 10, 20], n_users_sample=50)

print("\nNCF Ranking Results:")
for k, metrics in ranking_results_ncf.items():
    print(f"\nK={k}:")
    print(f"  Precision@{k}: {metrics['precision']:.4f}")
    print(f"  Recall@{k}: {metrics['recall']:.4f}")
    print(f"  NDCG@{k}: {metrics['ndcg']:.4f}")

In [ ]:
# Évaluer Two-Tower avec métriques de ranking
print("\n=== Évaluation Ranking Two-Tower ===")
ranking_results_tt = evaluate_ranking(two_tower_model, test_df, train_df, k_list=[5, 10, 20], n_users_sample=50)

print("\nTwo-Tower Ranking Results:")
for k, metrics in ranking_results_tt.items():
    print(f"\nK={k}:")
    print(f"  Precision@{k}: {metrics['precision']:.4f}")
    print(f"  Recall@{k}: {metrics['recall']:.4f}")
    print(f"  NDCG@{k}: {metrics['ndcg']:.4f}")

## 6. Comparaison Finale

In [ ]:
# Comparaison des modèles
comparison_df = pd.DataFrame({
    'Modèle': ['NCF', 'AutoRec', 'Two-Tower'],
    'RMSE': [test_rmse_ncf, test_rmse_ae, test_rmse_tt],
    'MAE': [test_mae_ncf, test_mae_ae, test_mae_tt]
})

print("\n=== Comparaison des Modèles (Rating Prediction) ===")
print(comparison_df.to_string(index=False))

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

models = comparison_df['Modèle']
x_pos = np.arange(len(models))

axes[0].bar(x_pos, comparison_df['RMSE'], color=['steelblue', 'coral', 'mediumseagreen'])
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(models)
axes[0].set_ylabel('RMSE')
axes[0].set_title('RMSE par Modèle')
axes[0].grid(axis='y', alpha=0.3)

axes[1].bar(x_pos, comparison_df['MAE'], color=['steelblue', 'coral', 'mediumseagreen'])
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(models)
axes[1].set_ylabel('MAE')
axes[1].set_title('MAE par Modèle')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualiser les métriques de ranking
k_values = [5, 10, 20]
ncf_precisions = [ranking_results_ncf[k]['precision'] for k in k_values]
ncf_recalls = [ranking_results_ncf[k]['recall'] for k in k_values]
ncf_ndcgs = [ranking_results_ncf[k]['ndcg'] for k in k_values]

tt_precisions = [ranking_results_tt[k]['precision'] for k in k_values]
tt_recalls = [ranking_results_tt[k]['recall'] for k in k_values]
tt_ndcgs = [ranking_results_tt[k]['ndcg'] for k in k_values]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Precision@K
axes[0].plot(k_values, ncf_precisions, marker='o', label='NCF', linewidth=2)
axes[0].plot(k_values, tt_precisions, marker='s', label='Two-Tower', linewidth=2)
axes[0].set_xlabel('K')
axes[0].set_ylabel('Precision@K')
axes[0].set_title('Precision@K')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Recall@K
axes[1].plot(k_values, ncf_recalls, marker='o', label='NCF', linewidth=2)
axes[1].plot(k_values, tt_recalls, marker='s', label='Two-Tower', linewidth=2)
axes[1].set_xlabel('K')
axes[1].set_ylabel('Recall@K')
axes[1].set_title('Recall@K')
axes[1].legend()
axes[1].grid(alpha=0.3)

# NDCG@K
axes[2].plot(k_values, ncf_ndcgs, marker='o', label='NCF', linewidth=2)
axes[2].plot(k_values, tt_ndcgs, marker='s', label='Two-Tower', linewidth=2)
axes[2].set_xlabel('K')
axes[2].set_ylabel('NDCG@K')
axes[2].set_title('NDCG@K')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Conclusion

Dans ce notebook, nous avons exploré les approches de **Deep Learning** pour les systèmes de recommandation :

1. **Neural Collaborative Filtering (NCF)** : combine embeddings + MLP pour capturer des interactions non-linéaires
2. **AutoRec** : autoencoder qui reconstruit les vecteurs de ratings utilisateurs
3. **Two-Tower Model** : architecture scalable avec deux tours séparées

**Observations** :
- Les modèles deep learning obtiennent généralement des performances comparables ou meilleures que SVD
- NCF et Two-Tower capturent mieux les interactions complexes
- AutoRec peut souffrir de la sparsité extrême de la matrice
- Les métriques de ranking (Precision@K, NDCG) sont plus pertinentes pour évaluer les recommandations top-K

**Avantages du Deep Learning** :
- Flexibilité pour incorporer des features supplémentaires (content, contexte)
- Meilleure capture des patterns non-linéaires
- Scalabilité avec les données massives

**Prochaines étapes** :
- Incorporer des features utilisateurs/items (hybrid models)
- Explorer les modèles séquentiels (GRU4Rec, SASRec) pour les sessions
- Déployer en production avec recherche ANN (FAISS) pour la scalabilité